In [3]:
import Functions as f
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split # typically done at the start of the script
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer
from sklearn.metrics import log_loss
from sklearn.linear_model import Ridge
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso


In [4]:
data = pd.read_csv('./data/funding_and_scores.csv')


In [5]:
#seperating test and train data
test_data = data[data['year'] == 2014]
training_data = data[data['year'] != 2014]
training_data.head()

,Unnamed: 0,State,year,Number.of.districts,Minimum,Maximum,Mean,score
0,1,Alabama,1995,127,5691657.6,405581866.2,4.101663e+07,0.701875
1,2,Alabama,2000,128,7786867.2,551756732.8,5.373821e+07,0.695625
2,3,Alabama,2005,131,6607702.2,661439354.4,5.463843e+07,0.704583
3,4,Alabama,2010,132,6317688.3,631356264.0,6.290127e+07,0.676250
5,6,Alaska,1995,54,1120001.4,600126575.4,3.462980e+07,0.646250


In [6]:
X_train = training_data.drop(['score', 'State', 'Number.of.districts', 'year' ], axis=1)
y_train = np.ravel(training_data.score)

In [7]:
train_features, test_features, train_outcome, test_outcome = train_test_split(X_train, y_train, test_size = 0.3, random_state = 11)


In [8]:
# normalize data
# train_features = f.pre_process(train_features)
# test_features = f.pre_process(test_features)

In [9]:
train_small_feature, validation_feature, train_small_outcome, validation_outcome = train_test_split(train_features, train_outcome, test_size = 0.2, random_state = 11)


In [10]:
clf = linear_model.Lasso(alpha= 0.1)

In [11]:
lasso_model = clf.fit(train_small_feature, train_small_outcome)

In [12]:
test_pred = clf.predict(validation_feature)

In [13]:
len(validation_outcome)

29

In [14]:
type(validation_outcome)

numpy.ndarray

In [17]:
type(test_pred)

numpy.ndarray

In [18]:
log_loss(validation_outcome, test_pred)

ValueError: Unknown label type: (array([ 0.624375  ,  0.71333333,  0.7525    ,  0.621875  ,  0.67125   ,
        0.631875  ,  0.60875   ,  0.63541667,  0.70375   ,  0.64708333,
        0.746875  ,  0.633125  ,  0.73833333,  0.62      ,  0.68      ,
        0.70125   ,  0.69458333,  0.73333333,  0.72416667,  0.626875  ,
        0.723125  ,  0.655     ,  0.63041667,  0.57958333,  0.625625  ,
        0.634375  ,  0.68708333,  0.6525    ,  0.60833333]),)

In [19]:
accuracy_score(validation_outcome, test_pred)

ValueError: continuous is not supported

In [ ]:
#### 

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
regr = ElasticNet()

param_grid_en = {'elasticnet__max_iter': np.arange(500, 2000),
                 'elasticnet__alpha': np.arange(0.05,0.2)}

reg = Lasso()
param_grid_ln ={'lasso__'} 
reg.get_params()

In [ ]:
grid = f.run_model(regr, param_grid_en,train_features, train_outcome)